In [14]:
import requests
import os
from dotenv import load_dotenv
from authlib.integrations.requests_client import OAuth2Session
import time

# Fetch credentials from environment
load_dotenv()
BLIZZARD_API_CLIENT_ID = os.getenv("BLIZZARD_API_CLIENT_ID")
BLIZZARD_API_CLIENT_SECRET = os.getenv("BLIZZARD_API_CLIENT_SECRET")
BLIZZARD_API_REGION = os.getenv("BLIZZARD_API_REGION").lower()

In [43]:
class BlizzardAuctionHouseClient:
    """Client for fetching World of Warcraft Auction House data from Battle.net API"""

    def __init__(self, client_id, client_secret, region="us", locale="en_US"):
        """
        Initialize the client with OAuth credentials
        
        Args:
            client_id: Your Battle.net API client ID
            client_secret: Your Battle.net API client secret
            region: API region ('us', 'eu', 'kr', 'tw', 'cn')
        """
        self.client_id = client_id
        self.client_secret = client_secret
        self.region = region.lower()
        
        # Set region-specific endpoints
        self.token_url = f'https://{self.region}.battle.net/oauth/token'
        self.api_base = f'https://{self.region}.api.blizzard.com'

        # Namespaces
        self.namespace_static = f"static-{self.region}"
        self.namespace_dynamic = f"dynamic-{self.region}"

        # Locale
        self.locale = locale
        
        # Initialize OAuth2 session
        self.session = OAuth2Session(
            client_id=self.client_id,
            client_secret=self.client_secret,
            token_endpoint=self.token_url
        )
        
        self.token = None
        self._authenticate()

        # HTTP default params
        self.default_params = {'namespace': self.namespace_dynamic, 'locale': self.locale}

    def _authenticate(self):
        """Authenticate and get access token using client credentials flow"""
        try:
            self.token = self.session.fetch_token(
                self.token_url,
                grant_type='client_credentials'
            )
            print(f"Successfully authenticated. Token expires in {self.token.get('expires_in')} seconds")
        except Exception as e:
            print(f"Authentication failed: {e}")
            raise

    def _ensure_valid_token(self):
        """Check if token is valid and refresh if necessary"""
        if not self.token or self.token.get('expires_at', 0) <= time.time():
            print("Token expired or missing, re-authenticating...")
            self._authenticate()

    def get_connected_realms(self):
        """
        Get the connected realms data.

        Returns:
            List of dictionaries containing all connected realms data of the region.
        """
        self._ensure_valid_token()
        
        url = f"{self.api_base}/data/wow/connected-realm/index"
        
        response_index = self.session.get(url, params=self.default_params)
        response_index.raise_for_status()

        realms = response_index.json().get('connected_realms')

        realms_data = []

        for realm in realms:
            response_data = self.session.get(realm.get("href"), params=self.default_params)
            response_data.raise_for_status()
            realms_data.append(response_data.json())

        return realms_data

    def get_auctions(self, connected_realm_id):
        """
        Get auction house data for a specific connected realm
        
        Args:
            connected_realm_id: The connected realm ID
        
        Returns:
            Dictionary containing auction house data
        """
        self._ensure_valid_token()
        
        url = f"{self.api_base}/data/wow/connected-realm/{connected_realm_id}/auctions"
        
        response = self.session.get(url, params=self.default_params)
        response.raise_for_status()
        return response.json()
    
    def get_commodity_auctions(self):
        """
        Get commodity auction house data (items that are region-wide)
        
        Returns:
            Dictionary containing commodity auction data
        """
        self._ensure_valid_token()
        
        url = f"{self.api_base}/data/wow/auctions/commodities"
        
        response = self.session.get(url, params=self.default_params)
        response.raise_for_status()
        return response.json()
    
client = BlizzardAuctionHouseClient(BLIZZARD_API_CLIENT_ID, BLIZZARD_API_CLIENT_SECRET, BLIZZARD_API_REGION)

Successfully authenticated. Token expires in 86399 seconds


In [44]:
# List realms
print("Fetching connected realms...")
realms = client.get_connected_realms()
print(f"Found {len(realms)} connected realms")

for realm in realms:
    print(realm)

Fetching connected realms...
Found 92 connected realms
{'_links': {'self': {'href': 'https://eu.api.blizzard.com/data/wow/connected-realm/1080?namespace=dynamic-eu'}}, 'id': 1080, 'has_queue': False, 'status': {'type': 'UP', 'name': 'Up'}, 'population': {'type': 'LOW', 'name': 'Low'}, 'realms': [{'id': 1080, 'region': {'key': {'href': 'https://eu.api.blizzard.com/data/wow/region/3?namespace=dynamic-eu'}, 'name': 'Europe', 'id': 3}, 'connected_realm': {'href': 'https://eu.api.blizzard.com/data/wow/connected-realm/1080?namespace=dynamic-eu'}, 'name': 'Khadgar', 'category': 'English', 'locale': 'enGB', 'timezone': 'Europe/Paris', 'type': {'type': 'NORMAL', 'name': 'Normal'}, 'is_tournament': False, 'slug': 'khadgar'}, {'id': 504, 'region': {'key': {'href': 'https://eu.api.blizzard.com/data/wow/region/3?namespace=dynamic-eu'}, 'name': 'Europe', 'id': 3}, 'connected_realm': {'href': 'https://eu.api.blizzard.com/data/wow/connected-realm/1080?namespace=dynamic-eu'}, 'name': 'Bloodhoof', 'cate